In [ ]:
import os
import sys
import logging

import boto3
import numpy as np
import pandas as pd
import sagemaker
import torch
from botocore.exceptions import ClientError

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = "eu-central-1"
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

In [ ]:
train_input_path = f"s3://quant-sagemaker-test-roles/imdb/data/small/train.csv"
test_input_path = f"s3://quant-sagemaker-test-roles/imdb/data/small/test.csv"

## Training

In [ ]:
from sagemaker.tensorflow import TensorFlow
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 32,
    "model_name": "distilbert-base-uncased",
}
estimator = TensorFlow(
    entry_point="train.py",
    source_dir="source",
    role=role,
    framework_version="2.4",
    py_version="py37",
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
)
estimator.fit({'train': train_input_path, 'test': test_input_path})

## Inference

In [ ]:
# model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-611215368770/tensorflow-training-2021-08-06-08-14-29-753/model.tar.gz'
print(model_data)

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel

model = TensorFlowModel(
    model_data=model_data,
    role=role,
    framework_version="2.4.1",
    source_dir="source",
    entry_point="inference.py",
)


### Endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="local",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.StringDeserializer(),
)

In [ ]:
input_1 = "this is a very good movie"
predictor.predict(input_1)

In [ ]:
input_2 = ['this movie sucks', 'this movie is ok']
predictor.predict(input_2)

In [ ]:
input_3 = ['such a terrible movie', 'what a great movie', 'omg best movie ever']
predictor.predict(input_3)

In [ ]:
predictor.delete_endpoint()

### Batch Transform

In [ ]:
transformer = model.transformer(instance_count=1, instance_type="local", accept='text/csv')

In [ ]:
transformer.transform(test_input_path, content_type='text/csv', split_type='Line')

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

In [ ]:
res = pd.read_csv(f"{transformer.output_path.split('/')[-1]}/test.csv.out", header=None)
res

### Local

In [ ]:
%%sh -s $estimator.model_data
rm -rf model
mkdir model
aws s3 cp $1 model/ 
tar xvzf model/model.tar.gz --directory ./model

In [ ]:
# pytorch_model.bin stores fine-tuned huggingface
# tokenizer_config.json stores tokenizer used for training

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')

In [ ]:
inputs = ['this is a terrific movie']

In [ ]:
tokenized_input = tokenizer(inputs, truncation=True, padding=True)

In [ ]:
input_ids = torch.Tensor(tokenized_input['input_ids']).long()
attention_mask = torch.Tensor(tokenized_input['attention_mask']).long()

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask).logits
    res = np.argmax(logits)
    print('logits:', logits, 'neg/pos:', res)